# Cube conversion to HDF5

This notebooks creates a `HDF5` version of each `SEG-Y` cube.

Pseudocode of this notebook looks like:

```python
for each cube:
    mkdir
    infer geometry
    if SHOW, log to std.out
    
    convert segy to hdf5
```

* The parameter `paths` controls which cubes are converted
* `RECREATE` determines whether already converted volumes are re-converted
* `SHOW` allows to control whether results are shown in the notebook itself
* `DRY` can be used to check which operations will happen, without actually executing them

In [ ]:
import os
import sys
import warnings
from tqdm.auto import tqdm
from glob import glob
import matplotlib.pyplot as plt

sys.path.append('..')
from seismiqb import SeismicGeometry, plot_image

In [ ]:
paths = sorted(glob('/data/seismic_data/seismic_interpretation/CUBE_*/amp*.sgy'))
[print(path) for path in paths]

RECREATE = False
SHOW = True

DRY_RUN = True

In [ ]:
%%time
for path_cube in tqdm(paths, ncols=1100):
    if not os.path.exists(path_cube):
        continue
    
    hdf5_path_cube = '.'.join((os.path.splitext(path_cube)[0], 'hdf5'))
    if os.path.exists(hdf5_path_cube) and not RECREATE:
        continue
    
    if DRY_RUN:
        print('Will convert :::', path_cube)
        continue
    
    if SHOW:
        print('▆'*55); print('▆'*55);
        print('Working with', path_cube)

    geometry = SeismicGeometry(
        path_cube,
        headers=SeismicGeometry.HEADERS_POST_FULL,
        index_headers = SeismicGeometry.INDEX_POST,
        collect_stats=True, spatial=True, recollect=True
    )
    qmap = geometry.quality_map

    if SHOW:
        print(geometry)
        plot_image(qmap, cmap='Reds')
        plt.show()
    
    geometry.make_hdf5()